## DATASET CLEANING AND TRANSFORM

In [3]:
PARAMETERS = {
	'BALANCE_RATIO': 0.35,
	'DOWNLOAD_FROM_DRIVE': False,
	'INSTALL_LIBS': False,
	'CLASS_COL_NAME': 'Protestas',
	'START_YEAR': 2016,
	'BALANCED_OUPUT_FILE_NAME': 'balanced_output_35.csv',
	'TEST_2023_FILE_NAME': 'test_2023.csv',
}

In [4]:
if(PARAMETERS['INSTALL_LIBS']):
	%pip install -U imbalanced-learn
	%pip install scikit-learn==1.0.2

In [5]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) #MOSTRAR TODAS LAS COLUMNAS
y_col_name = PARAMETERS['CLASS_COL_NAME']

In [6]:
if(PARAMETERS['DOWNLOAD_FROM_DRIVE']):
	!gdown --id 1-tNBUCWFU0_b9QW6ie4Vvfs7eb4WbIPY
	# https://drive.google.com/file/d/1-tNBUCWFU0_b9QW6ie4Vvfs7eb4WbIPY/view?usp=share_link
	!gdown --id 11eGwBDC3FoJh8eanfpYXFAS-3SZ6vNnO
	#https://drive.google.com/file/d/11eGwBDC3FoJh8eanfpYXFAS-3SZ6vNnO/view?usp=sharing

In [7]:
df_completo = pd.read_csv('TestBalanceo1.csv', sep=";;;", index_col=False)
df_banderas_completo = pd.read_csv('OcidConBanderas.csv', sep=";", index_col=False)
df_banderas_completo.drop(['Unnamed: 10'], inplace=True, axis=1)
df_completo = df_completo.merge(df_banderas_completo, left_on='ocid', right_on='id', how='left')

/var/folders/0s/ngx4k9892vv59rf4q5h1h4w00000gn/T/ipykernel_3669/2578874532.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_completo = pd.read_csv('TestBalanceo1.csv', sep=";;;", index_col=False)


In [8]:
df_completo_filtered = df_completo[df_completo['tender.value.amount'].notna()]
df_completo_filtered = df_completo_filtered[df_completo_filtered['tender.tenderPeriod.durationInDays'].notna()]
df_completo_filtered = df_completo_filtered[df_completo_filtered['tender.enquiryPeriod.durationInDays'].notna()]
# df_dummizado['tender.tenderPeriod.durationInDays'].fillna(df_dummizado['tender.tenderPeriod.durationInDays'].mean(), inplace=True)
# df_dummizado['tender.enquiryPeriod.durationInDays'].fillna(df_dummizado['tender.enquiryPeriod.durationInDays'].mean(), inplace=True)

In [9]:
#Drop columns
df_dummizado = df_completo_filtered.drop(['ocid', 'id_x', 'id_y', 'tender.id', # Dropeamos por que son IDs
    'tender.bidOpening.address.streetAddress', 'tender.enquiriesAddress.streetAddress', #Dropeamos por que son direcciones, creemos que no tienen valor
    'ocid.1', 'tender.procurementIntention.uri', # URL y id
    'tender.procurementIntention.description', #Descripcion
    'tender.procurementIntention.category', # Dropeamos porque ya esta la info en otra columna
    'tender.procurementIntention.title', #Titulo, practicamente todos son unicos
    'initiationType', #Todos tienen el mismo valor (tender)
    'tender.procurementIntention.id', # Ver vien por que dropeamos
    'tag', #Todos tienen el mismo valor (compiled)
    'language', #Todos tienen el mismo valor (es)
    'tender.techniques.hasFrameworkAgreement', #Solo 48 rows con datos, no vale la pena
    # 'Protestas', #Tiene info del resultado
	'tender.contractPeriod.durationInDays',
	'has_complaint',
], axis=1)

df_dummizado['date.month'] = pd.to_datetime(df_dummizado['date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['date.year'] = pd.to_datetime(df_dummizado['date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['date.yearmonth'] = pd.to_datetime(df_dummizado['date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

In [10]:
df_dummizado = df_dummizado[df_dummizado['date.year'] >= PARAMETERS['START_YEAR']] #JUST USING 2016 onwards

In [11]:
#Dummies
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.status']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.awardCriteria']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.awardCriteriaDetails']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.statusDetails']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.hasEnquiries']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.value.currency']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.mainProcurementCategory']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.procurementMethod']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.procurementMethodRationale']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.procurementIntention.rationale']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['planning.budget.amount.currency']) # Cardinalidad baja

df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.procurementIntention.status']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.procurementIntention.statusDetails']) # Cardinalidad baja


#Dates
df_dummizado['tender.bidOpening.date.month'] = pd.to_datetime(df_dummizado['tender.bidOpening.date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.bidOpening.date.year'] = pd.to_datetime(df_dummizado['tender.bidOpening.date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.bidOpening.date.yearmonth'] = pd.to_datetime(df_dummizado['tender.bidOpening.date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['tender.datePublished.month'] = pd.to_datetime(df_dummizado['tender.datePublished'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.datePublished.year'] = pd.to_datetime(df_dummizado['tender.datePublished'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.datePublished.yearmonth'] = pd.to_datetime(df_dummizado['tender.datePublished'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['tender.tenderPeriod.startDate.month'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.tenderPeriod.startDate.year'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.tenderPeriod.startDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['tender.tenderPeriod.endDate.month'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.tenderPeriod.endDate.year'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.tenderPeriod.endDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['tender.awardPeriod.startDate.month'] = pd.to_datetime(df_dummizado['tender.awardPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.awardPeriod.startDate.year'] = pd.to_datetime(df_dummizado['tender.awardPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.awardPeriod.startDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.awardPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['tender.enquiryPeriod.endDate.month'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.endDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.enquiryPeriod.endDate.year'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.endDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.enquiryPeriod.endDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.endDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['tender.enquiryPeriod.startDate.month'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.enquiryPeriod.startDate.year'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.enquiryPeriod.startDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')

df_dummizado['planning.estimatedDate.month'] = pd.to_datetime(df_dummizado['planning.estimatedDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['planning.estimatedDate.year'] = pd.to_datetime(df_dummizado['planning.estimatedDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['planning.estimatedDate.yearmonth'] = pd.to_datetime(df_dummizado['planning.estimatedDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%Y%m')
# Dropeando data interesante (Despues ver como procesar, son datos practicamente unicos)
df_dummizado = df_dummizado.drop(['tender.title', 'tender.submissionMethodDetails', 'tender.eligibilityCriteria', 'tender.mainProcurementCategoryDetails', 'planning.budget.description', ], axis=1)

# Dropeando por cardinalidad alta? Datos no 100% unicos por cada row
df_dummizado = df_dummizado.drop(['tender.procurementMethodDetails', 'tender.procuringEntity.id', 'tender.procuringEntity.name', 'buyer.id', 'buyer.name', 
	'tender.procurementIntention.procuringEntity.id', 'tender.procurementIntention.procuringEntity.name', 'secondStage.id',
], axis=1)


In [12]:
#Dropeando dates
df_dummizado = df_dummizado.drop(['tender.bidOpening.date',
	'tender.datePublished', 'tender.tenderPeriod.startDate', 'tender.tenderPeriod.endDate', 'tender.awardPeriod.startDate','tender.enquiryPeriod.endDate','tender.enquiryPeriod.startDate',
    'date', 'planning.estimatedDate', 
    'tender.contractPeriod.maxExtentDate', # Hay solo 7000
    'tender.procurementIntention.startDate', 'tender.procurementIntention.publishedDate', #Hay solo 1200 en todo el dataset
	'tender.contractPeriod.startDate', 'tender.contractPeriod.endDate' # Hay solo 35 datos en todo el dataset
], axis= 1)

# Otras conversiones
df_dummizado['tender.techniques.hasElectronicAuction'] = np.where(df_dummizado['tender.techniques.hasElectronicAuction'] == True, 1, 0)

In [13]:
df_dummizado['tender.bidOpening.date.month'].fillna(0, inplace=True)
df_dummizado['tender.bidOpening.date.year'].fillna(0, inplace=True)
df_dummizado['tender.bidOpening.date.yearmonth'].fillna(0, inplace=True)

df_dummizado['tender.enquiryPeriod.endDate.month'].fillna(0, inplace=True)
df_dummizado['tender.enquiryPeriod.endDate.year'].fillna(0, inplace=True)
df_dummizado['tender.enquiryPeriod.endDate.yearmonth'].fillna(0, inplace=True)
df_dummizado['tender.enquiryPeriod.startDate.month'].fillna(0, inplace=True)
df_dummizado['tender.enquiryPeriod.startDate.year'].fillna(0, inplace=True)
df_dummizado['tender.enquiryPeriod.startDate.yearmonth'].fillna(0, inplace=True)
df_dummizado['tender.tenderPeriod.startDate.month'].fillna(0, inplace=True)
df_dummizado['tender.tenderPeriod.startDate.year'].fillna(0, inplace=True)
df_dummizado['tender.tenderPeriod.startDate.yearmonth'].fillna(0, inplace=True)
df_dummizado['tender.tenderPeriod.endDate.month'].fillna(0, inplace=True)
df_dummizado['tender.tenderPeriod.endDate.year'].fillna(0, inplace=True)
df_dummizado['tender.tenderPeriod.endDate.yearmonth'].fillna(0, inplace=True)
df_dummizado['tender.awardPeriod.startDate.month'].fillna(0, inplace=True)
df_dummizado['tender.awardPeriod.startDate.year'].fillna(0, inplace=True)
df_dummizado['tender.awardPeriod.startDate.yearmonth'].fillna(0, inplace=True)
df_dummizado['tender.datePublished.month'].fillna(0, inplace=True)
df_dummizado['tender.datePublished.year'].fillna(0, inplace=True)
df_dummizado['tender.datePublished.yearmonth'].fillna(0, inplace=True)
df_dummizado['planning.estimatedDate.month'].fillna(0, inplace=True)
df_dummizado['planning.estimatedDate.year'].fillna(0, inplace=True)
df_dummizado['planning.estimatedDate.yearmonth'].fillna(0, inplace=True)
df_dummizado['planning.identifier'] = df_dummizado['planning.identifier'].fillna(0)

df_dummizado['tender.numberOfTenderers'].fillna(1, inplace=True)
df_dummizado['tender.techniques.hasElectronicAuction'] = df_dummizado['tender.techniques.hasElectronicAuction'].fillna(False)
# df_dummizado['tender.value.amount'].fillna(df_dummizado['tender.value.amount'].mean(), inplace=True)

df_dummizado['planning.budget.amount.amount'].fillna(df_dummizado['tender.value.amount'], inplace=True)


In [14]:
df_dummizado_up_to_2023 = df_dummizado[df_dummizado['date.year'] < 2023]
df_dummizado_up_to_2023.shape

(89675, 785)

In [15]:
df_dummizado_from_2023 = df_dummizado[df_dummizado['date.year'] >= 2023]
df_dummizado_from_2023.shape

(10769, 785)

In [16]:
df_complaints = df_dummizado_up_to_2023[(df_dummizado_up_to_2023[y_col_name] == True)]
df_no_complaints = df_dummizado_up_to_2023[(df_dummizado_up_to_2023[y_col_name] == False)]
# PERCENTAGE OF COMPLAINTS / NO COMPLAINTS
(df_complaints.shape[0] * 1) / df_no_complaints.shape[0]

0.10763206976198417

## SMOTE BALANCING (FOR)

In [30]:
import h2o
from h2o.estimators import H2OXGBoostEstimator
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 hours 47 mins
H2O_cluster_timezone:,America/Asuncion
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.1
H2O_cluster_version_age:,1 month and 19 days
H2O_cluster_name:,H2O_from_python_matiaslopez_nsue9n
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,14.13 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [23]:
def get_performance(model, test: h2o.H2OFrame):
	cm = model.model_performance(test).confusion_matrix().to_list()
	perf_t = cm[1][1] / (cm[1][0] + cm[1][1])
	perf_f = cm[0][0] / (cm[0][1] + cm[0][0])
	return perf_t, perf_f

In [27]:
# BEST SMOTE HYPERPARAMS:
# H2OXGBoostEstimator - 0.8228451418286427 => 0.155

In [26]:
from imblearn.over_sampling import SMOTE
from collections import Counter
X = df_dummizado_up_to_2023.drop(y_col_name, axis=1)  # Features (all columns except 'target')
y = df_dummizado_up_to_2023[y_col_name]  # Target variable
print('Original dataset shape %s' % Counter(y))

best_value = 0
best_aucpr = 0

for value in [0.11, 0.125, 0.14, 0.155, 0.17, 0.185, 0.2, 0.215, 0.23, 0.245, 0.26, 0.275, 0.29, 0.305, 0.32, 0.335, 0.35]:
  #https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html#examples-using-imblearn-over-sampling-smote
  smote = SMOTE(random_state=24, sampling_strategy= value)
  X_resampled, y_resampled = smote.fit_resample(X, y)
  print('para value {}:'.format(value))
  resampled_df = pd.concat([X_resampled, y_resampled], axis=1, join="inner")
  
  # desde acá cada run del algoritmo

  train = h2o.H2OFrame.from_python(resampled_df)
  valid = h2o.H2OFrame.from_python(df_dummizado_from_2023)

  titanic_xgb = H2OXGBoostEstimator(booster='dart',
                                    stopping_metric= 'AUCPR',
                                    normalize_type="tree",
                                    seed=1234)
  titanic_xgb.train(x=train.columns,
                    y='Protestas',
                    training_frame=train,
                    validation_frame=valid)

  if (best_aucpr < titanic_xgb.aucpr(valid=True)):
    best_aucpr = titanic_xgb.aucpr(valid=True)
    bes_value = value

  print(titanic_xgb.aucpr(valid=True))
  print(get_performance(titanic_xgb, valid))
print("best value: {} - best aucpr {}".format(best_value, best_aucpr))

Original dataset shape Counter({False: 80961, True: 8714})


/usr/local/lib/python3.10/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


para value 0.11:
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost Model Build progress: |

/usr/local/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: Dropping bad and constant columns: [tender.procurementMethod_open, parties.details.legalEntityTypeDetail buyer_15, parties.details.legalEntityTypeDetail buyer_14, parties.details.legalEntityTypeDetail buyer_13, parties.details.legalEntityTypeDetail buyer_12, parties.details.legalEntityTypeDetail buyer_11, parties.details.legalEntityTypeDetail buyer_10, parties.details.EntityType payee_1, parties.details.EntityType payee_2, parties.details.EntityType payee_3, parties.details.EntityType payee_4, planning.items.classification.id.n1_1_57, planning.items.classification.id.n1_1_58, Monto faltante, tender.status_active, tender.statusDetails_En Convocatoria (Abierta), parties.details.EntityType candidate_2, parties.details.EntityType candidate_1, parties.details.EntityType candidate_4, parties.details.legalEntityTypeDetail candidate_1, awards.statusDetails_2, parties.details.EntityType candidate_3, pa

██████████████████████████████████████████████████| (done) 100%
0.8203102609097221
(0.7429775280898876, 0.9084711526322434)


...

██████████████████████████████████████████████████| (done) 100%
0.8122369919327221
(0.7535112359550562, 0.8931953036232799)


/usr/local/lib/python3.10/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


para value 0.35:
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost Model Build progress: |

/usr/local/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: Dropping bad and constant columns: [tender.procurementMethod_open, parties.details.legalEntityTypeDetail buyer_15, parties.details.legalEntityTypeDetail buyer_14, parties.details.legalEntityTypeDetail buyer_13, parties.details.legalEntityTypeDetail buyer_12, parties.details.legalEntityTypeDetail buyer_11, parties.details.legalEntityTypeDetail buyer_10, parties.details.EntityType payee_1, parties.details.EntityType payee_2, parties.details.EntityType payee_3, parties.details.EntityType payee_4, planning.items.classification.id.n1_1_57, planning.items.classification.id.n1_1_58, Monto faltante, tender.status_active, tender.statusDetails_En Convocatoria (Abierta), parties.details.EntityType candidate_2, parties.details.EntityType candidate_1, parties.details.EntityType candidate_4, parties.details.legalEntityTypeDetail candidate_1, awards.statusDetails_2, parties.details.EntityType candidate_3, pa

██████████████████████████████████████████████████| (done) 100%
0.8188876075154826
(0.7205056179775281, 0.9202120944325212)
best value: 0 - best aucpr 0.8228451418286427
